In [2]:
using BenchmarkTools
using Dates
using ProgressBars
using Distances

In [1]:
include("../src/foldexity.jl")
include("../src/entropy.jl")

In [12]:
fxpdb("../testpdb/AF-Q8VCK6-F1-model_v4.pdb", 6, 1.0) 


Starting foldexity...


(4.667813588592014, 4.13240610986081, 96, 0.09696969696969697, 990, [0.0 1.879364013671875 … 3.222787380218506 1.3873801231384277; 1.879364013671875 0.0 … 3.5461854934692383 2.0417065620422363; … ; 3.222787380218506 3.5461854934692383 … 0.0 3.1773934364318848; 1.3873801231384277 2.0417065620422363 … 3.1773934364318848 0.0])

In [16]:
fxdir("../testpdb", "fxdata.tsv", 6, 1.0)
df = CSV.read("fxdata.tsv", DataFrame, delim = "\t")
rm("fxdata.tsv")

5 files are collected 
Starting foldexity...


20.0%┣████████▋                                  ┫ 1/5 [00:00<Inf:Inf, InfGs/it]
60.0%┣████████████████████████████▉                   ┫ 3/5 [00:03<00:03, 2s/it]
80.0%┣██████████████████████████████████████▍         ┫ 4/5 [00:06<00:02, 2s/it]
100.0%┣███████████████████████████████████████████████┫ 5/5 [00:06<00:00, 2s/it]


In [6]:
pdb = readpdb_backbone("../testpdb/AF-Q8VCK6-F1-model_v4.pdb")
writepdb(pdb, "../testpdb/backbone.pdb")
fsize = 8

xyzcoords = pdb2xyz(pdb)
xyzcoords = coords2fragments(xyzcoords, fsize)

pdbmatrix = pdb2pdbmatrix(pdb)
pdbfragments = coords2fragments(pdbmatrix, fsize)

fxity, aver_rmsd, nclusts, norm_nclusts, nfrags, matrix = fxity_kabsh(xyzcoords)

(7.309983070517202, 8.600171516053804, 220, 0.6811145510835913, 323, [0.0 12.18087387084961 … 20.524507522583008 20.470008850097656; 12.18087387084961 0.0 … 22.167490005493164 22.168933868408203; … ; 20.524507522583008 22.167490005493164 … 0.0 4.060173988342285; 20.470008850097656 22.168933868408203 … 4.060173988342285 0.0])

In [22]:
# dump PDB fragments
pdb = readpdb_backbone("../testpdb/small.pdb")
pdbfragments = coords2fragments(pdbmatrix, )
pdbfragments = coords2fragments(pdbmatrix, 20)

for (i, frag) in enumerate(pdbfragments)
    writepdb(pdbmatrix2pdb(frag), "../frags/frag$i.pdb")
end

In [7]:
pfrag1 = xyzcoords[10]
pfrag2 = xyzcoords[11]

kabsch_umeyama(pfrag1, pfrag2)

0.36361846f0